In [1]:
import numpy as np
import os
from skimage.metrics import structural_similarity as ssim
from autoencoder_compressor import Compressor_Decompressor
import ae
%reload_ext autoreload
%autoreload 2

In [5]:
def get_ssim(original_image,retrieved_image):
    return ssim(original_image,retrieved_image,axis=2)

In [2]:
def get_mse(original_image,retrieved_image):
    #Mean Square Error
    diff = original_image-retrieved_image
    return (np.square(diff)).mean(axis=None)

In [3]:
def get_psnr(original_image,retrieved_image):
    #Peak signal to noise ratio
    mse = get_mse(original_image,retrieved_image)
    max = 255
    return 20*np.log10(max)-10*np.log10(mse)

In [6]:
metrics = {'mse':get_mse,
           'psnr':get_psnr,
           'ssim':get_ssim
}

In [ ]:
def get_compression_stats(model,image_path):
    c , image_size = model.compress_image(image_path)
    d = model.decompress_image(c,image_path_out,image_size,return_image=True)
    res = {}
    for metric_key in metrics:
        res[metric_key] = metrics[metric_key](c,d)
    return res


In [ ]:
def test_model(models_root:str,model_name:str,image_list:list,compression_out:int,model_type=ae.Autoencoder_5hidden):
    model_path = os.path.join(models_root,model_name)
    res = {'name':model_name}
    model = Compressor_Decompressor(model_path,ae.Autoencoder_5hidden,compression_out)
    results_per_image = []
    for image_path in image_list:
        results_per_image.append(get_compression_stats(model,image_path))
    for metric_key in metrics:
        metric_result = np.asarray([image_result[metric_key] for image_result in results_per_image])
        res[metric_key] = metric_result.mean(axis=None)
    return res #{'name':'ae...','mse':23,'psnr':200,'ssim':0.8}

In [ ]:
def test_all_models(models_root:str,model_name_dict:dict):
    metrics = []
    for model_name in model_name_dict:
        

In [4]:

wd_path = os.path.abspath(os.getcwd())
model_path = os.path.join(wd_path, "models","ae_5_10_0.pt")
image_path = os.path.join(wd_path,"images","testing","misc","5.1.13.tiff")
image_path_out = os.path.join(wd_path,"images","5.1.13_out.tiff")

comp = Compressor_Decompressor(model_path,ae.Autoencoder_5hidden,compression_out=8)
c , image_size = comp.compress_image(image_path)

res = comp.decompress_image(c,image_path_out,image_size,return_image=True)

original = Compressor_Decompressor.load_image_as_np(image_path)
print(original.nbytes)
print(np.asarray(c).nbytes)
print("Ratio = ", original.nbytes/np.asarray(c).nbytes)
print("MSE :" ,get_mse(original,res))
print("PSNR:" ,get_psnr(original,res))
print("SSIM:" ,ssim(original,res,channel_axis=2))

196608
16384
Ratio =  12.0
MSE : 76.15682474772136
PSNR: 29.313715324149495
SSIM: 0.8228937425419258
